In [0]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost

In [0]:
import pandas as pd
import numpy as np

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import eli5
from eli5.sklearn import PermutationImportance

In [13]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [14]:
# Data read
df = pd.read_hdf("data/car.h5")
df.shape

(106494, 155)

In [0]:
# factorize all features
SUFFIX_CAT = "__cat"

for feat in df.columns:
  if isinstance(df[feat][0], list): continue
    
  factorized_values = df[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df[feat] = factorized_values
  else:
    df[feat + SUFFIX_CAT] = factorized_values

In [16]:
cat_feats = [x for x in df.columns if SUFFIX_CAT in x]
cat_feats = [x for x in cat_feats if "price" not in x]
len(cat_feats)

151

In [0]:
def run_model (model, feats):
  X = df[feats].values
  y = df['price_value'].values

  scores = cross_val_score(model, X, y, cv=3, scoring="neg_mean_absolute_error")

  return np.mean(scores), np.std(scores)

Decission tree

In [25]:
run_model(DecisionTreeRegressor(max_depth=5), cat_feats)

(-19695.13091100928, 148.72570644015792)

Random forest

In [26]:
run_model(
    RandomForestRegressor(max_depth=5,
                          n_estimators=50,
                          random_state = 0),
    cat_feats)

(-18718.657185256638, 64.5424578125788)

XGBoost

In [27]:
xgb_params = {
    "max_depth": 5,
    "n_estimators": 50,
    "learning_rate": 0.1,
    "seed": 0
}

model = xgb.XGBRegressor(**xgb_params)

run_model(model, cat_feats)

[20:05:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:05:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:06:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13108.379065811214, 74.32158265003798)

In [28]:
xgb_params = {
    "max_depth": 5,
    "n_estimators": 50,
    "learning_rate": 0.1,
    "seed": 0
}
m = xgb.XGBRegressor(**xgb_params)
m.fit(X,y)

imp = PermutationImportance(m, random_state = 0).fit(X,y)
eli5.show_weights(imp, feature_names=cat_feats)

[20:08:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1194 ± 0.0031,param_napęd__cat
0.1132 ± 0.0032,param_rok-produkcji__cat
0.1090 ± 0.0025,param_stan__cat
0.0619 ± 0.0025,param_skrzynia-biegów__cat
0.0568 ± 0.0016,param_faktura-vat__cat
0.0489 ± 0.0014,param_moc__cat
0.0273 ± 0.0007,param_marka-pojazdu__cat
0.0242 ± 0.0013,feature_kamera-cofania__cat
0.0212 ± 0.0008,param_typ__cat
0.0174 ± 0.0008,param_pojemność-skokowa__cat


In [44]:
# take top 20 features in dataset
important_feats = eli5.formatters.as_dataframe.explain_weights_df(imp, feature_names=cat_feats)
feats = important_feats[0:20].feature
feats = feats.tolist()
feats

['param_napęd__cat',
 'param_rok-produkcji__cat',
 'param_stan__cat',
 'param_skrzynia-biegów__cat',
 'param_faktura-vat__cat',
 'param_moc__cat',
 'param_marka-pojazdu__cat',
 'feature_kamera-cofania__cat',
 'param_typ__cat',
 'param_pojemność-skokowa__cat',
 'seller_name__cat',
 'feature_wspomaganie-kierownicy__cat',
 'param_model-pojazdu__cat',
 'param_wersja__cat',
 'param_kod-silnika__cat',
 'feature_system-start-stop__cat',
 'feature_asystent-pasa-ruchu__cat',
 'feature_czujniki-parkowania-przednie__cat',
 'feature_łopatki-zmiany-biegów__cat',
 'feature_regulowane-zawieszenie__cat']

In [45]:
xgb_params = {
    "max_depth": 5,
    "n_estimators": 50,
    "learning_rate": 0.1,
    "seed": 0
}

model = xgb.XGBRegressor(**xgb_params)

run_model(model, feats)

[20:27:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:27:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:27:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13375.230420852275, 65.40441107118909)

In [53]:
for f in feats:
   # remove the suffix to plot real features and not vectorized ones
   f = (f[:-5])
   print(f)
   print(df[f].unique())

param_napęd
[None 'Na przednie koła' '4x4 (dołączany automatycznie)' 'Na tylne koła'
 '4x4 (dołączany ręcznie)' '4x4 (stały)']
param_rok-produkcji
['2018' '2011' '2015' '2009' '2017' '2012' '2013' '2007' '2001' '2016'
 '2006' '2008' '2004' '1999' '2000' '2010' '2005' '2002' '1998' '2014'
 '2003' '1982' '1995' '1997' '1992' '1993' '1994' '1996' '1989' '1988'
 '1967' '1987' '1970' '1959' '1990' '1991' '1974' None '1975' '1973'
 '1953' '1985' '1984' '1986' '1981' '1979' '1960' '1983' '1978' '1964'
 '1980' '1972' '1969' '1956' '1966' '1977' '1962' '1965' '1971' '1963'
 '1961' '1952' '1949' '1976' '1937' '1968' '1958' '1955' '1933' '1929'
 '1957' '1944' '1954' '1932' '1936' '1947' '1948']
param_stan
['Nowe' 'Używane' None]
param_skrzynia-biegów
['Manualna' 'Automatyczna hydrauliczna (klasyczna)'
 'Automatyczna dwusprzęgłowa (DCT, DSG)' None
 'Automatyczna bezstopniowa (CVT)' 'Półautomatyczna (ASG, Tiptronic)']
param_faktura-vat
[None 'Tak']
param_moc
['90 KM' '115 KM' '262 KM' '110 KM' '310

There are some featurs that are numeric but we use them vectorised, lets try to use their original values

In [0]:
# param_rok-produkcji
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == "None" else int(x))

In [68]:
feats[1] = 'param_rok-produkcji'

xgb_params = {
    "max_depth": 5,
    "n_estimators": 50,
    "learning_rate": 0.1,
    "seed": 0
}

model = xgb.XGBRegressor(**xgb_params)

run_model(model, feats)

[20:45:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:45:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:45:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-11308.885890938496, 27.868488259630677)

In [0]:
# param_moc '1 116 KM'
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == "None" else int(x.replace(" ", "").strip("KM")))

In [107]:
feats[5] = 'param_moc'

xgb_params = {
    "max_depth": 5,
    "n_estimators": 50,
    "learning_rate": 0.1,
    "seed": 0
}

model = xgb.XGBRegressor(**xgb_params)

run_model(model, feats)

[21:02:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:02:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:02:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9754.814038220027, 78.76301552059402)

In [0]:
# param_pojemność-skokowa '1 560 cm3'
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == "None" else int(x.split("cm3")[0].replace(" ", "")))

In [128]:
feats[9] = 'param_pojemność-skokowa'

xgb_params = {
    "max_depth": 5,
    "n_estimators": 50,
    "learning_rate": 0.1,
    "seed": 0
}

model = xgb.XGBRegressor(**xgb_params)

run_model(model, feats)

[21:10:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[21:10:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9621.119663721702, 100.59307679134791)